In [1]:
import os
os.chdir("..")

In [2]:
import hydra
from omegaconf import DictConfig

from shell.fleet.helper import get_fleet, get_agent_cls
import torch
import time
import datetime
import logging
from shell.utils.experiment_utils import setup_experiment, process_dataset_cfg, eval_net
from shell.datasets.datasets import get_dataset
from pprint import pprint
import ray
import os
from hydra import compose, initialize
from omegaconf import OmegaConf
from shell.utils.utils import seed_everything
from shell.fleet.model_sharing_utils import is_in
logging.basicConfig(level=logging.INFO)

In [3]:
# config_path = os.path.join("conf", "grad.yaml")
# # read the config file
# cfg = omegaconf.OmegaConf.load(config_path)
initialize(config_path="conf", job_name="tmp_job")
cfg = compose(config_name="grad")
seed_everything(cfg.seed)

/tmp/ipykernel_1902582/4114385434.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf", job_name="tmp_job")
/home/vlongle/miniconda3/envs/shell/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'grad': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
# override
cfg.parallel = True

In [5]:
cfg.train.num_epochs = 50
cfg.train.component_update_freq = 50

In [6]:
AgentCls = get_agent_cls(cfg.sharing_strategy, cfg.algo, cfg.parallel)

graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(
        cfg)
FleetCls = get_fleet(cfg.sharing_strategy, cfg.parallel)

INFO:root:Class sequence: [4 7 0 8 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)


{'train': {'component_update_freq': 50, 'num_epochs': 50, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'gradient', 'num_coms_per_round': 5, 'retrain': {'num_epochs': 10}}, 'seed': 0, 'algo': 'modular', 'job_name': 'fun', 'num_agents': 4, 'root_save_dir': 'results', 'parallel': True, 'num_init_tasks': 2, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05, 'use_contrastive': True}}


INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [2 1 3 8 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 9 7 2 1 8 6 1 6 4 5 7 8 0 2 3 0 3 9 6]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 5 3 4 4 5 9 3 0 6 9 1 3 1 7 9 5 2 3 2]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(12

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 2}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


In [7]:
fake_dataset = get_dataset(**process_dataset_cfg(cfg))
fleet = FleetCls(graph, cfg.seed, datasets, cfg.sharing_strategy, AgentCls, NetCls=NetCls,
                     LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
                     train_kwargs=train_cfg, fake_dataset=fake_dataset)

INFO:root:Class sequence: [3 9 2 4 9 2 5 4 8 9 9 4 1 6 9 6 8 0 5 1]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
2023-03-27 22:16:19,007	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ParallelModGrad pid=1910811) INFO:root:Agent: node_id: 69420, seed: 69420000
(ParallelModGrad pid=1910811) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1910811) INFO:root:	task: 0	loss: 6.208	acc: 0.500
(ParallelModGrad pid=1910811) INFO:root:	task: avg	loss: 6.208	acc: 0.500
(ParallelModGrad pid=1910811) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1910811) INFO:root:	task: 0	loss: 6.201	acc: 0.500
(ParallelModGrad pid=1910811) INFO:root:	task: avg	loss: 6.201	acc: 0.500
(ParallelModGrad pid=1910811) INFO:root:f

In [8]:
def diff_models(modelA_statedict, modelB_statedict, keys=None):
    diffs = {}
    # compute the average difference between two models
    for key in modelA_statedict.keys():
        if keys is not None and not is_in(key, keys):
            continue
        if key in modelB_statedict.keys():
            diffs[key] = torch.mean(torch.abs(modelA_statedict[key] - modelB_statedict[key])).item()
    return diffs

In [9]:
# diff_models(fleet.agents[0].get_net().state_dict(), fleet.agents[1].get_net().state_dict(),
#             keys=["random_linear_projection", "components"])

diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

(ParallelModGrad pid=1970633) replacing model with strict: False
(ParallelModGrad pid=1970632) replacing model with strict: False
(ParallelModGrad pid=1970681) replacing model with strict: False
(ParallelModGrad pid=1970755) replacing model with strict: False


{'components.0.weight': 0.0,
 'components.0.bias': 0.0,
 'components.1.weight': 0.0,
 'components.1.bias': 0.0,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [10]:
ray.get(fleet.agents[0].eval.remote(0))

{0: 0.4900497512437811}

In [11]:
# check the linear projection
print("agent 0:", ray.get(fleet.agents[0].get_net.remote()).state_dict()["random_linear_projection.weight"][0, :10])
print("agent 1:", ray.get(fleet.agents[1].get_net.remote()).state_dict()["random_linear_projection.weight"][0, :10])

agent 0: tensor([ 0.0232, -0.0111,  0.0021, -0.0189, -0.0179, -0.0112, -0.0022,  0.0112,
        -0.0076,  0.0167], device='cuda:0')
agent 1: tensor([ 0.0232, -0.0111,  0.0021, -0.0189, -0.0179, -0.0112, -0.0022,  0.0112,
        -0.0076,  0.0167], device='cuda:0')


In [12]:
print("agent 0:", ray.get(fleet.agents[0].get_net.remote()).state_dict()["random_linear_projection.weight"].requires_grad)

agent 0: False


In [13]:
# local train
fleet.train(0)

(ParallelModGrad pid=1970632) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1970632) INFO:root:	task: 0	loss: 6.188	acc: 0.490
(ParallelModGrad pid=1970632) INFO:root:	task: avg	loss: 6.188	acc: 0.490
(ParallelModGrad pid=1970633) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1970633) INFO:root:	task: 0	loss: 6.229	acc: 0.524
(ParallelModGrad pid=1970633) INFO:root:	task: avg	loss: 6.229	acc: 0.524
(ParallelModGrad pid=1970681) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1970681) INFO:root:	task: 0	loss: 6.142	acc: 0.493
(ParallelModGrad pid=1970681) INFO:root:	task: avg	loss: 6.142	acc: 0.493
(ParallelModGrad pid=1970755) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=1970755) INFO:root:	task: 0	loss: 6.345	acc: 0.475
(ParallelModGrad pid=1970755) INFO:root:	task: avg	loss: 6.345	acc: 0.475
(ParallelModGrad pid=1970632) INFO:root:epochs: 1, training task: 0
(ParallelModGrad pid=1970632) INFO:root:	task: 0	loss: 6.211	acc: 0.520
(Par

In [14]:
ray.get(fleet.agents[0].eval.remote(0))

{0: 0.9238805970149254}

In [15]:
for agent in fleet.agents:
    net = ray.get(agent.get_net.remote())
    print('component.0.bias', net.state_dict()["components.0.bias"][0])

component.0.bias tensor(-0.0153, device='cuda:0')
component.0.bias tensor(-0.0227, device='cuda:0')
component.0.bias tensor(-0.0226, device='cuda:0')
component.0.bias tensor(-0.0199, device='cuda:0')


In [16]:
(-0.0153-0.0227-0.0226-0.0199)/4

-0.020125

In [17]:
(-0.0220-0.0227-0.0226-0.0199)/4

-0.0218

In [18]:
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

{'components.0.weight': 0.0028567069675773382,
 'components.0.bias': 0.002630018861964345,
 'components.1.weight': 0.0027251215651631355,
 'components.1.bias': 0.002240207977592945,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [19]:
fleet

In [20]:
fleet.num_coms_per_round = 1
fleet.communicate(0)

(ParallelModGrad pid=1970633) INFO:root:node 1 is communicating at round 0 for task 0
(ParallelModGrad pid=1970633) INFO:root:My neighbors are: [Actor(ParallelModGrad, 65a2d58fed038567f5080d6d01000000), Actor(ParallelModGrad, b55b9e3a726aaebe9fb72fb701000000), Actor(ParallelModGrad, a6cf07f4e66c81055fa72df601000000)]
(ParallelModGrad pid=1970632) INFO:root:node 0 is communicating at round 0 for task 0
(ParallelModGrad pid=1970632) INFO:root:My neighbors are: [Actor(ParallelModGrad, 2ec33562626d501ec80d6b3501000000), Actor(ParallelModGrad, b55b9e3a726aaebe9fb72fb701000000), Actor(ParallelModGrad, a6cf07f4e66c81055fa72df601000000)]
(ParallelModGrad pid=1970681) INFO:root:node 2 is communicating at round 0 for task 0
(ParallelModGrad pid=1970681) INFO:root:My neighbors are: [Actor(ParallelModGrad, 65a2d58fed038567f5080d6d01000000), Actor(ParallelModGrad, 2ec33562626d501ec80d6b3501000000), Actor(ParallelModGrad, a6cf07f4e66c81055fa72df601000000)]
(ParallelModGrad pid=1970755) INFO:root:nod

In [21]:
# check the linear projection
print("agent 0:", ray.get(fleet.agents[0].get_net.remote()).state_dict()["random_linear_projection.weight"][0, :10])
print("agent 1:", ray.get(fleet.agents[1].get_net.remote()).state_dict()["random_linear_projection.weight"][0, :10])

agent 0: tensor([ 0.0232, -0.0111,  0.0021, -0.0189, -0.0179, -0.0112, -0.0022,  0.0112,
        -0.0076,  0.0167], device='cuda:0')
agent 1: tensor([ 0.0232, -0.0111,  0.0021, -0.0189, -0.0179, -0.0112, -0.0022,  0.0112,
        -0.0076,  0.0167], device='cuda:0')


In [22]:
for agent in fleet.agents:
    net = ray.get(agent.get_net.remote())
    print('component.0.bias', net.state_dict()["components.0.bias"][0])

(ParallelModGrad pid=1970633) stuff_added: defaultdict(<class 'int'>, {'components.0.weight': 3, 'components.0.bias': 3, 'components.1.weight': 3, 'components.1.bias': 3})
(ParallelModGrad pid=1970632) stuff_added: defaultdict(<class 'int'>, {'components.0.weight': 3, 'components.0.bias': 3, 'components.1.weight': 3, 'components.1.bias': 3})
(ParallelModGrad pid=1970681) stuff_added: defaultdict(<class 'int'>, {'components.0.weight': 3, 'components.0.bias': 3, 'components.1.weight': 3, 'components.1.bias': 3})
(ParallelModGrad pid=1970755) stuff_added: defaultdict(<class 'int'>, {'components.0.weight': 3, 'components.0.bias': 3, 'components.1.weight': 3, 'components.1.bias': 3})
component.0.bias tensor(-0.0201, device='cuda:0')
component.0.bias tensor(-0.0201, device='cuda:0')
component.0.bias tensor(-0.0201, device='cuda:0')
component.0.bias tensor(-0.0201, device='cuda:0')


In [23]:
# ray.get(fleet.agents[0].get_bytes_sent.remote())

In [24]:
# ray.get(fleet.agents[0].get_received_models.remote())[2]['components.0.weight']

In [25]:
# ray.get(fleet.agents[1].get_received_models.remote())[2]['components.0.weight']

In [26]:
# # before aggregation
# ray.get(fleet.agents[0].get_model.remote())['components.0.weight']

In [27]:
# ray.get(fleet.agents[0].aggregate_models.remote())
# fleet.process_communicate(0)

In [28]:
# # make sure that this doesn't affect agent 1
# ray.get(fleet.agents[1].get_received_models.remote())[2]['components.0.weight']

In [29]:
# # after aggregation
# ray.get(fleet.agents[0].get_model.remote())['components.0.weight']

In [30]:
# evaluate the performance again
ray.get(fleet.agents[0].eval.remote(0))

{0: 0.8880597014925373}

In [33]:
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

{'components.0.weight': 0.0,
 'components.0.bias': 0.0,
 'components.1.weight': 0.0,
 'components.1.bias': 0.0,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [34]:
# check the linear projection
ray.get(fleet.agents[0].get_net.remote()).state_dict()["random_linear_projection.weight"][0, :10]

tensor([ 0.0232, -0.0111,  0.0021, -0.0189, -0.0179, -0.0112, -0.0022,  0.0112,
        -0.0076,  0.0167], device='cuda:0')